# Interpreter Activity 2
## 17:00 Period

In [1]:
data Val = IntVal Integer
         | BoolVal Bool
         | Closure String Exp Env
  deriving (Show,Eq)
  
data Exp = IntExp Integer
         | IntOpExp String Exp Exp
         | VarExp String
         | LetExp String Exp Exp
         | FunExp String Exp
         | AppExp Exp Exp
  deriving (Show,Eq)
  
type Env = [(String,Val)]

In [2]:
intOps = [ ("+",(+))
         , ("-",(-))
         , ("*",(*))
         , ("/",div)]

In [3]:
liftIntOp f (IntVal i1) (IntVal i2) = IntVal (f i1 i2)
liftIntOp f _           _           = IntVal 0

In [4]:
tErr s = error $ "You idiot! " ++ s ++ " Sad!"

In [5]:
eval :: Exp -> Env -> Val
eval (IntExp i) _ = IntVal i
eval (IntOpExp op e1 e2) env =
  let v1 = eval e1 env
      v2 = eval e2 env
      Just f = lookup op intOps
   in liftIntOp f v1 v2 
eval (VarExp v) env =
  case lookup v env of
     Just vv -> vv
     Nothing -> tErr $ "Symbol " ++ v ++ " is undefined."
eval (LetExp v e1 e2) env =
  let v1 = eval e1 env
   in eval e2 ((v,v1):env)
eval (FunExp v body) env =
   Closure v body env
eval (AppExp e1 e2) env =
   let v1 = eval e1 env
       v2 = eval e2 env
    in case v1 of
        Closure p body clenv  ->  eval body ((p,v2):clenv)
        _                     -> tErr "That was not a closure!"

In [6]:
nats n = n : nats (n + 1)

In [7]:
take 10 (nats 10)

[10,11,12,13,14,15,16,17,18,19]

In [8]:
(\ x -> x * x + 1) 100

10001

In [9]:
foo = 
   let delta = 1 in
      \x -> x * x + delta

In [10]:
map (\x -> x*x+1) [3,4,5,7]

[10,17,26,50]

In [11]:
p1 = LetExp "delta" (IntExp 1) (FunExp "x" (IntOpExp "+" (VarExp "x") (VarExp "delta"))) 

In [12]:
eval p1 []

Closure "x" (IntOpExp "+" (VarExp "x") (VarExp "delta")) [("delta",IntVal 1)]

In [14]:
p2 = LetExp "foo" p1 (AppExp (VarExp "foo") (IntExp 10))

In [15]:
eval p2 []

IntVal 11

In [16]:
twice = FunExp "f" (FunExp "x" (AppExp (VarExp "f") (AppExp (VarExp "f") (VarExp "x"))))

In [18]:
eval (AppExp (AppExp (VarExp "twice") (VarExp "foo")) (IntExp 10)) [("foo",eval p1 []),("twice",eval twice [])]

IntVal 12